In [2]:
##########################################
############                  ############
############    MUST HAVES    ############
############                  ############
##########################################

print("Must Haves")


Must Haves


In [4]:
# Read CSV
# This will read in the CSVs that come from the alpha vantage website, where each CSV
# is from a separate stock symbol
import csv  

def open_csv(path_to_csv):
    data = []
    with open(path_to_csv) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0] != '':
                data.append(row)
    return data


In [5]:
# Access API
from myapikey import APIkey
from alpha_vantage.timeseries import TimeSeries


In [6]:
# must provide an api key (they are free from the alpha vantage website
# this determines how much data you can get in how much time.
def get_intraday(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas', indexing_type='integer')
    return ts.get_intraday(symbol=stock, interval='1min', outputsize='full')

# provide an api key to access the data
def get_daily(stock):
    ts = TimeSeries(key=APIkey, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=stock, outputsize='full')
    return data, meta_data


In [7]:
# Select stocks based on ticker symbol
stock_names = ["AAOI", "AAP", "AAPL", "ABMD", "ABT", "ACET", "ADIL", "AETI", "AFL", "AKAM", "ALGN", "ALQA", "AMD", 
"AMG", "AMRH", "AMZN", "ANY", "APVO", "AVGR", "AXSM", "BAC", "BIIB", "BLIN", "BLK", "BLRX", "BSQR", "BSX", "BTAI", 
"CGA", "CGC", "CLIR", "CMCSA", "CMG", "COP", "COTY", "COUP", "CPSH", "CSCO", "CTK", "CTRV", "CVX", "D", "DIS", 
"DUK", "EXC", "EYES", "FARO", "FB", "FCEL", "FDS", "FLKS", "FRAN", "FTNT", "GE", "GM", "GNMX", "GOOGL", "HMC", "HOV", 
"HSGX", "IGLD", "INOD", "IVZ", "JAGX", "JNJ", "JPM", "JT", "KEYS", "KOSS", "LB", "LLY", "LPTH", "M", "MDT", "MSFT", 
"MYSZ", "NDAQ", "NEE", "NFLX", "NKE", "NRG", "NVDA", "NVFY", "ORLY", "OSS", "OXBR", "PFE", "PIR", "PRGO", "QBAK", 
"QUIK", "RAD", "RBCN","RHT", "RL", "SBUX", "SESN", "SFUN", "SGMA", "SMAR", "SO", "STZ", "SYK", "T", "TNK", "TOPS", 
"TRHC", "TRIL", "TRIP", "TRNX", "TSLA", "TSN", "TTWO", "TWMC", "TWTR", "UA", "UAA", "UPL", "USB", "UXIN", "V", 
"VRSN", "VTVT", "VZ", "WDC", "WFC", "WFT", "WMT", "XOM", "ZTS"]

# at least one of these is not a real stock recognized by alpha vantage.
# I must error check these stocks.
print(len(stock_names))


130


In [13]:
# Store Stock information as a data set
stock_data = batch_get_daily(stock_names)


In [3]:
# print(stock_data[0][0])
import pickle


In [14]:
f = open('store_stocks_new.pckl', 'wb') # rename and remove '_new' to load it
pickle.dump(stock_data, f)
f.close()


In [4]:
f = open('store_stocks.pckl', 'rb')
stock_data = pickle.load(f)
f.close()


In [15]:
# Format dataset with classification (up/down)
import numpy as np

def add_classification(stocks):
    # up = 1, down = 0
    new_stocks = []
    # column 4 is the end of day price
    last_price = stocks[len(stocks) - 1][4]
    # the very last price will always be up. Perhaps
    # this makes my AI optimistic?
    new_stocks.append(np.insert(stocks[len(stocks) - 1], 5, 1)) # up
    for s in reversed(list(range(len(stocks) - 1))):
        if (stocks[s][4] - last_price) > 0:
            new_stocks.append(np.insert(stocks[s], 5, 1)) # up
        else:
            new_stocks.append(np.insert(stocks[s], 5, 0)) # down
        last_price = stocks[s][4]

    return new_stocks


In [29]:
print(add_classification(stock_data[0][0].to_numpy()))
one_stock = [[0, 1, 2, 3, 1, 0], [1, 2, 3, 4, 1, 0]] #...
sample = []
for start in range(len(one_stock) - 50):
    sample.append(one_stock[start:(start + 50)])


[array([8.92000e+00, 9.16000e+00, 8.89100e+00, 9.03000e+00, 2.97033e+05,
       1.00000e+00]), array([8.94000e+00, 9.03000e+00, 8.72000e+00, 8.86000e+00, 8.05473e+05,
       1.00000e+00]), array([9.42000e+00, 9.46000e+00, 9.00000e+00, 9.08000e+00, 5.45325e+05,
       0.00000e+00]), array([9.46000e+00, 9.71000e+00, 9.41000e+00, 9.52000e+00, 5.84264e+05,
       1.00000e+00]), array([9.03000e+00, 9.43000e+00, 8.96000e+00, 9.27000e+00, 6.80875e+05,
       1.00000e+00]), array([9.57000e+00, 9.62390e+00, 9.16000e+00, 9.17000e+00, 9.97801e+05,
       1.00000e+00]), array([9.98000e+00, 1.01000e+01, 9.70000e+00, 9.71000e+00, 6.14007e+05,
       0.00000e+00]), array([1.02500e+01, 1.04300e+01, 9.99000e+00, 1.00100e+01, 6.01777e+05,
       0.00000e+00]), array([1.0250e+01, 1.0531e+01, 1.0190e+01, 1.0380e+01, 6.0023e+05,
       0.0000e+00]), array([1.03700e+01, 1.04500e+01, 9.91740e+00, 1.01600e+01, 8.30895e+05,
       1.00000e+00]), array([1.08100e+01, 1.10600e+01, 1.03050e+01, 1.07100e+01, 9.6519

In [9]:
# (3 must-haves in one cell)
# Train
# Hidden Nodes
# Dropout
import tensorflow as tf

# example training
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)


NameError: name 'x_train' is not defined

In [ ]:
# Test
model.evaluate(x_test, y_test)


In [ ]:
# Real Life Prediction


In [ ]:
# Print results of prediction


In [ ]:
##########################################
#######                           ########
#######    WOULD LIKE TO HAVES    ########
#######                           ########
##########################################

print("would like to haves")


In [ ]:
# Save data as CSV



In [12]:
# Wait for API (free) access limit
import time

def batch_get_daily(stock_list):
    count = 0
    data = []
    for stock in stock_list:
        if count == 5:
            time.sleep(65) # 65 seconds just in case the timing on the server or here isn't perfect
            count = 0
        data.append(get_daily(stock))
        count += 1
    return data

# for 130 stocks this should take at least half an hour


In [ ]:
# Save results of Training
# https://www.tensorflow.org/guide/keras#entire_model
model.save_weights('verdurouMKI.h5')


In [ ]:
# Load Results from Previous Training
model = tf.keras.models.load_model('verdurouMKI.h5')


In [22]:
# Scrape HTML
